In [8]:
# Get dependencies
import requests
import pandas as pd

# load bigfoot data
bigfoot_coords = pd.read_json('../data/bigfoot_coordinates_clean_cols.json')

,report_number,report_class,state,county,latitude,longitude,season,month,observed
0,13038,A,Alaska,Anchorage County,61.119996,-149.745430,Winter,February,I and two of my friends were bored one night s...
1,8792,B,Alaska,Anchorage County,61.217576,-149.899678,Winter,December,"Me and a couple of friends had been bored, whe..."
2,637,A,Alaska,Cordova-McCarthy County,61.486389,-142.886389,Summer,June,My hiking partner and I arrived late to the Ke...
3,26604,A,Alaska,Fairbanks County,64.840051,-147.719976,Summer,July,"It was the month of July, 2009 in Fairbanks Al..."
4,1995,B,Alaska,Fairbanks County,64.840051,-147.719976,Fall,August,This happened in late August of 1997 in a side...


In [9]:
county_df = bigfoot_coords[['county', 'state']].drop_duplicates(subset=['county','state'])

In [24]:
# API URL and params
api_url = "https://api.census.gov/data/2020/acs/acs5"
params = {
    "get": "NAME,B01003_001E",
    "for": "county:*",
    "key": "257ae49299267c8d6f1e7e156233669b61cb73b1"
}

# Make the API request
response = requests.get(api_url,params=params)

# check for successful connection
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Create a DataFrame
    columns = data[0] # Column names
    rows = data[1:] # Data rows
    county_pop_df = pd.DataFrame(rows, columns=columns)

    # Cleanup  
    county_pop_df.rename(columns={
        "NAME": "name", 
        "B01003_001E": "population", 
        "state": "state_no",
        "county": "county_no" 
        }, inplace=True)
    county_pop_df['population'] = pd.to_numeric(county_pop_df['population'], errors='coerce')
    
    print(county_pop_df.head())


                      name  population state_no county_no
0  Autauga County, Alabama       55639       01       001
1  Baldwin County, Alabama      218289       01       003
2  Barbour County, Alabama       25026       01       005
3     Bibb County, Alabama       22374       01       007
4   Blount County, Alabama       57755       01       009


In [27]:
county_pop_df[['county', 'state']] = county_pop_df['name'].str.split(',', expand=True)
county_pop_df.head()
county_pop_df = county_pop_df[['state', 'county', 'population']]
county_pop_df.to_json('../data/county_populations.json')